<a href="https://colab.research.google.com/github/satyam0402/python-ml/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

STEP 1 : WORK FLOW

1. Fetching News Data dataset

2. Data Pre processing ( imp step : To convert text to numbers )

3. Train Test Split the data

4. Feeding the split data to ML model for training the model ( Using logistic regression as its binary classification problem as we are classifing the result as real or fake )

5. Evaluation of Trained model ( We will find accuracy score of the model using test data)

6. Feeding the new data.

7. Result : Real news or fake news






STEP 2 : Importing and understanding the News data set.



In [66]:
import pandas as pd
news_dtf = pd.read_csv('/content/News.csv')
display(news_dtf.head())
display(news_dtf.info())

,Unnamed: 0,title,text,subject,date,class
0,0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44919 entries, 0 to 44918
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  44919 non-null  int64 
 1   title       44919 non-null  object
 2   text        44919 non-null  object
 3   subject     44898 non-null  object
 4   date        44898 non-null  object
 5   class       44919 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 2.1+ MB


None

STEP 3 : Importing Dependencies

In [67]:
import pandas as pd
import numpy as np
import re # re : regular expression used for searching text in doc.
from nltk.corpus import stopwords # here corpus is body of the text/content and nltk is natural language text kit. Stopwords are those words that does not add much value to our data. Words like : The , A , were, what etc.
from nltk.stem.porter import PorterStemmer # here we will use function stemming in which it takes a word and remove the prefix and sufix of that word and returns the root word.
from sklearn.feature_extraction.text import TfidfVectorizer #  This finction converts the text into feature vector that is number.
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [68]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [69]:
# What are these stopwords ?
print(stopwords.words('english')) # Duriging stemming process we will remove these stopwords.

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

STEP 4 : Data Pre-Processing

STEP 4.1 : Loading the dataset to a panda dataframe

In [70]:
news_dtf = pd.read_csv('/content/News.csv')

In [71]:
# Rows and Columns of dataframe.
news_dtf.shape



(44919, 6)

In [72]:
news_dtf.head()

,Unnamed: 0,title,text,subject,date,class
0,0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [73]:
news_dtf.tail()

,Unnamed: 0,title,text,subject,date,class
44914,21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
44915,21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
44916,21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
44917,21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
44918,21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


STEP 4.2 : Counting the number of missing values in the dataset

In [74]:
# Missing values are not important so we can count and replace it with null.
news_dtf.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
subject,21
date,21
class,0


NOTE : If lot of values are missing then in that case we have to replace the missing values with appropriate values and this method is called as IMPUTATION.

STEP 4.3 : Replacing the null values with the empty string.

In [75]:
news_data = news_dtf.fillna('')


STEP 4.4 : Merging title and text data.

In [76]:
news_data['content'] = news_data['title']

In [77]:
print(news_data['content'])

0         Donald Trump Sends Out Embarrassing New Year’...
1         Drunk Bragging Trump Staffer Started Russian ...
2         Sheriff David Clarke Becomes An Internet Joke...
3         Trump Is So Obsessed He Even Has Obama’s Name...
4         Pope Francis Just Called Out Donald Trump Dur...
                               ...                        
44914    'Fully committed' NATO backs new U.S. approach...
44915    LexisNexis withdrew two products from Chinese ...
44916    Minsk cultural hub becomes haven from authorities
44917    Vatican upbeat on possibility of Pope Francis ...
44918    Indonesia to buy $1.14 billion worth of Russia...
Name: content, Length: 44919, dtype: object


STEP 4.5 : Seperating data and label

In [78]:
X = news_data.drop(columns='class', axis = 1)
Y = news_data['class']
print (X)
print (Y)

       Unnamed: 0                                              title  \
0               0   Donald Trump Sends Out Embarrassing New Year’...   
1               1   Drunk Bragging Trump Staffer Started Russian ...   
2               2   Sheriff David Clarke Becomes An Internet Joke...   
3               3   Trump Is So Obsessed He Even Has Obama’s Name...   
4               4   Pope Francis Just Called Out Donald Trump Dur...   
...           ...                                                ...   
44914       21412  'Fully committed' NATO backs new U.S. approach...   
44915       21413  LexisNexis withdrew two products from Chinese ...   
44916       21414  Minsk cultural hub becomes haven from authorities   
44917       21415  Vatican upbeat on possibility of Pope Francis ...   
44918       21416  Indonesia to buy $1.14 billion worth of Russia...   

                                                    text    subject  \
0      Donald Trump just couldn t wish all Americans ...       N

4.6 : Stemming (Process of reducing word to root word)
example : Root word for actor, actress, acting is act.

In [79]:
prorter_stemmer = PorterStemmer() # Loading the PorterStemmer function to variable porter_stemmer.


In [80]:
# Defining function
def stemming(content) :
  porter_stemmer = PorterStemmer() # Loading the PorterStemmer function to variable porter_stemmer.
  stemmed_content = re.sub('[^a-zA-Z]',' ', content) # Here we are removing the number and just keeping the words and albhabets in content section (Title and text) , If there is number or anything it will be repcaed by ' '.
  stemmed_content = stemmed_content.lower() # Convert to lower case.
  stemmed_content = stemmed_content.split() # Split and convert to list.
  stemmed_content = [porter_stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # We are taking each word and stemming it, Passign it to for loop and only taking the word without the stopwords
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [81]:
news_data['content'] = news_data['content'].apply(stemming)

In [82]:
print(news_data['content'])

0        donald trump send embarrass new year eve messa...
1        drunk brag trump staffer start russian collus ...
2        sheriff david clark becom internet joke threat...
3            trump obsess even obama name code websit imag
4            pope franci call donald trump christma speech
                               ...                        
44914    fulli commit nato back new u approach afghanistan
44915         lexisnexi withdrew two product chines market
44916                        minsk cultur hub becom author
44917      vatican upbeat possibl pope franci visit russia
44918              indonesia buy billion worth russian jet
Name: content, Length: 44919, dtype: object


STEP 4.7 : Seperating the data and label

In [89]:
X = news_data['content'].values
Y = news_data['class'].values

In [84]:
print(X)
print(Y)
print(X.shape)
print(Y.shape)

       Unnamed: 0                                              title  \
0               0   Donald Trump Sends Out Embarrassing New Year’...   
1               1   Drunk Bragging Trump Staffer Started Russian ...   
2               2   Sheriff David Clarke Becomes An Internet Joke...   
3               3   Trump Is So Obsessed He Even Has Obama’s Name...   
4               4   Pope Francis Just Called Out Donald Trump Dur...   
...           ...                                                ...   
44914       21412  'Fully committed' NATO backs new U.S. approach...   
44915       21413  LexisNexis withdrew two products from Chinese ...   
44916       21414  Minsk cultural hub becomes haven from authorities   
44917       21415  Vatican upbeat on possibility of Pope Francis ...   
44918       21416  Indonesia to buy $1.14 billion worth of Russia...   

                                                    text    subject  \
0      Donald Trump just couldn t wish all Americans ...       N

STEP 5 : Converting textual data to numerical data

In [85]:
vectorizer = TfidfVectorizer() # Tf term frequence (counts frequency of words.) idf : Inverse document frequency ( check for words frequency and if of no value removes it)
vectorizer.fit(X)
X = vectorizer.transform(X)

In [86]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6 stored elements and shape (6, 6)>
  Coords	Values
  (0, 5)	1.0
  (1, 4)	1.0
  (2, 3)	1.0
  (3, 2)	1.0
  (4, 1)	1.0
  (5, 0)	1.0


STEP 7 : Splitting the data into training and test data

In [90]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 2)

STEP 8 : Training the model

In [92]:
model = LogisticRegression()

In [93]:
model.fit(X_train, Y_train)

ValueError: could not convert string to float: 'watch black live matter berni sander support spit u flag front vet trump ralli'

STEP 7 : Converting textual data to numerical data

In [94]:
vectorizer = TfidfVectorizer() # Tf term frequence (counts frequency of words.) idf : Inverse document frequency ( check for words frequency and if of no value removes it)
vectorizer.fit(X)
X = vectorizer.transform(X)

STEP 8 : Splitting the data into training and test data

In [95]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 2)

STEP 9 : Training the model

In [96]:
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

STEP 10 : Evaluation ( Accuracy score )

In [97]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [98]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.959482398775567


In [99]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [100]:
print('Accuracy score of the training data : ', test_data_accuracy)

Accuracy score of the training data :  0.9428984861976848


STEP 11 : Making prediction system

In [106]:
X_new_news = X_test[5]

prediction = model.predict(X_new_news)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [107]:
print(Y_test[5])

0
